In [6]:
from langchain_aws import ChatBedrockConverse
from langchain.schema import HumanMessage, SystemMessage
import boto3
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from opensearchpy import OpenSearch, RequestsHttpConnection
import os
import boto3
import json
import sys
from langchain.schema import BaseRetriever, Document
from typing import List, Optional, Dict, Tuple
from langchain.prompts.prompt import PromptTemplate
import uuid
from datetime import datetime
from botocore.exceptions import ClientError
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from boto3.dynamodb.conditions import Key
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_aws import ChatBedrock


In [7]:
print("langchain advanced")

langchain advanced


In [8]:
#bedrock client 설정
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-west-2'  # replace with your region
)

In [9]:
#llm
llm = ChatBedrock(
    client=bedrock_client,
    model_id="global.anthropic.claude-sonnet-4-20250514-v1:0",
    model_kwargs={
        "max_tokens": 2000,
        "temperature": 0.15,
        "top_p": 0.9,
    }
    )

In [10]:
async def extract_stream(chain, input_json):
    collected_text = []
    async for event in chain.astream_events(input_json):
        if event["event"] == "on_chat_model_stream":
            chunk = event["data"]["chunk"]
            if chunk.content:
                for content_item in chunk.content:
                    if text := content_item.get('text'):
                        collected_text.append(text)
                        print(text, end="", flush=True)
    return "".join(collected_text)


In [11]:
#input { json 포맷 }  -> prompt(text로 변환) -> llm(bedrock) -> output(string타입, json타입)
#여기서 input { json 포맷 }을 활용하는 방법에 대해 이해해보자 ( 여행에 관한 정보를 얻는 llm를 만든다고 가정한다 )

In [12]:
prompt_capital = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{country}의 수도에 대해 알려줘")
])

capital_chain = prompt_capital | llm |  StrOutputParser()


capital_chain.invoke({"country" : "태국"})

'태국의 수도는 **방콕(Bangkok)**입니다.\n\n## 방콕에 대한 주요 정보:\n\n### 기본 정보\n- **정식 명칭**: 크룽텝마하나콘 (กรุงเทพมหานคร)\n- **인구**: 약 1,000만 명 (수도권 포함)\n- **면적**: 약 1,569㎢\n\n### 주요 특징\n- **정치·경제 중심지**: 태국의 정치, 경제, 문화의 중심\n- **관광 명소**: 왕궁, 왓 포 사원, 왓 아룬 사원, 카오산 로드 등\n- **교통**: 스카이트레인(BTS), 지하철(MRT), 보트 등 다양한 교통수단\n- **음식**: 태국 전통 음식과 길거리 음식의 천국\n\n### 문화적 특징\n- 전통 태국 문화와 현대 문명이 공존\n- 불교 사원과 현대적 쇼핑몰이 함께 위치\n- 차오프라야 강이 도시를 관통하며 "동양의 베니스"라고도 불림\n\n방콕은 동남아시아의 주요 관광지이자 경제 허브로서 매년 수많은 관광객들이 방문하는 도시입니다.'

In [13]:
prompt_culture = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{country}의 문화에 대해 알려줘")
])

culture_chain = prompt_culture | llm |  StrOutputParser()


culture_chain.invoke({"country" : "태국"})


'태국의 풍부한 문화에 대해 소개해드리겠습니다.\n\n## 🏛️ 종교와 철학\n- **불교**: 인구의 95%가 소승불교도\n- 일상생활에 불교 철학이 깊이 스며들어 있음\n- 사원(왓)이 지역 공동체의 중심 역할\n\n## 🙏 예의와 매너\n- **와이(Wai)**: 손바닥을 모아 인사하는 전통 예법\n- 연장자와 승려에 대한 깊은 존경\n- 왕실에 대한 절대적 존경 (불경죄 엄격)\n- 머리는 신성한 부위로 여겨 함부로 만지지 않음\n\n## 🍜 음식 문화\n- **쌀**이 주식 (카오 = 밥, 식사를 의미)\n- 매운맛, 신맛, 단맛, 짠맛의 조화\n- 대표 음식: 똠얌꿍, 팟타이, 그린커리, 망고스티키라이스\n- 길거리 음식 문화 발달\n\n## 🎭 예술과 전통\n- **전통 무용**: 콘(Khon), 라콘(Lakhon) 등 가면극\n- **무에타이**: 전통 격투기\n- 정교한 사원 건축과 불상 조각\n- 전통 공예: 실크, 도자기, 목각\n\n## 🎉 축제와 기념일\n- **송크란**: 태국 신정 (4월, 물축제)\n- **로이 끄라통**: 등불 축제 (11월)\n- **부처님 오신 날**, **왕실 기념일** 등\n\n## 👨\u200d👩\u200d👧\u200d👦 사회 문화\n- 가족 중심의 사회\n- 계층 의식이 존재하지만 온화한 성격\n- "마이 펜 라이" (괜찮다) 정신 - 관용과 여유\n- 미소의 나라로 불릴 만큼 친근하고 온화\n\n태국 문화는 전통과 현대가 조화롭게 어우러진 매력적인 특징을 가지고 있습니다.'

In [14]:
prompt_climate = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{country}의 날씨에 대해 알려줘")
])

climate_chain = prompt_climate | llm |  StrOutputParser()


climate_chain.invoke({"country" : "태국"})

'태국의 날씨에 대해 설명드리겠습니다.\n\n## 기본 특징\n- **열대성 기후**: 연중 고온다습한 날씨\n- **평균 기온**: 25-35°C\n- **습도**: 연중 높은 습도 (70-80%)\n\n## 계절별 특징\n\n### 🌞 건기 (11월-2월)\n- 가장 여행하기 좋은 시기\n- 기온: 25-30°C\n- 습도가 상대적으로 낮음\n- 맑은 날씨가 많음\n\n### 🔥 더운 건기 (3월-5월)\n- 가장 더운 시기\n- 기온: 30-40°C\n- 매우 덥고 건조함\n- 4-5월이 가장 더움\n\n### 🌧️ 우기 (6월-10월)\n- 몬순 시즌\n- 기온: 25-32°C\n- 거의 매일 스콜성 비\n- 9-10월이 강우량 최고점\n\n## 지역별 차이\n- **북부** (치앙마이): 건기와 우기가 뚜렷\n- **중부** (방콕): 연중 고온, 우기 강우량 많음\n- **남부** (푸켓, 코사무이): 지역에 따라 우기 시기가 다름\n\n## 여행 팁\n- **최적 시기**: 11월-2월\n- **준비물**: 우산, 선크림, 가벼운 옷\n- **주의사항**: 우기에는 홍수 가능성 확인'

In [15]:
prompt_location = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{country}의 유명한 관광 명소에 대해 알려줘")
])

climate_location = prompt_location | llm |  StrOutputParser()


climate_location.invoke({"country" : "태국"})

"태국의 주요 관광 명소들을 지역별로 소개해드리겠습니다.\n\n## 🏛️ 방콕 (Bangkok)\n- **왓 포 (Wat Pho)** - 거대한 와불상으로 유명한 사원\n- **왓 아룬 (Wat Arun)** - 새벽 사원으로 불리는 아름다운 사원\n- **그랜드 팰리스** - 태국 왕실의 궁전\n- **차오프라야 강** - 롱테일 보트 투어\n- **카오산 로드** - 배낭여행자들의 성지\n\n## 🏖️ 푸켓 (Phuket)\n- **파통 비치** - 가장 유명한 해변\n- **피피 섬** - 영화 '더 비치' 촬영지\n- **카론 비치, 카타 비치** - 조용하고 아름다운 해변들\n\n## 🌴 끄라비 (Krabi)\n- **라일레이 비치** - 석회암 절벽으로 둘러싸인 해변\n- **홍 섬** - 에메랄드빛 바다\n- **타이거 케이브 템플** - 1,237개 계단을 올라가는 사원\n\n## 🏝️ 코사무이 (Koh Samui)\n- **차웽 비치** - 가장 인기 있는 해변\n- **빅 부다 템플** - 거대한 황금 불상\n\n## 🏛️ 치앙마이 (Chiang Mai)\n- **도이 수텝 사원** - 산 위의 황금 사원\n- **치앙마이 구시가지** - 고대 성벽과 전통 사원들\n- **나이트 바자** - 전통 수공예품 쇼핑\n\n## 🏺 아유타야 (Ayutthaya)\n- **아유타야 역사공원** - UNESCO 세계문화유산\n- **왓 마하탓** - 나무뿌리에 감싸인 불상 머리\n\n각 지역마다 독특한 매력이 있으니, 관심 있는 특정 지역이나 활동이 있다면 더 자세히 알려드릴 수 있습니다!"

In [16]:
#여행에 관한 정보를 얻는 llm를 만든다고 가정한다 위의 네가지 정보를 모두 포함하여 전체적인 형태의 여행 정보를 얻고 싶다고 가정 할때 어떻게 langchain에 활용할수 있을까?
#결국 llm에서 결과가 나오면 string임으로 이 string을 json안에 assgin가능하다

In [17]:
# input에 {"country": "태국"}이라고 두고 itemgetter("country")를 통해 변수를 가져온뒤 각각 llm의 결과를 얻고 sting으로 json에 포함시킨다면? 
# llm을 통과한 결과값들이 새로운 input(json타입)을 형성할수 있지 않을까?
new_input = {
                "capital" :  itemgetter("country") | capital_chain,
                "culture" :  itemgetter("country") | culture_chain,
                "climate" :  itemgetter("country") | climate_chain,
                "location":  itemgetter("country") | climate_chain
            }

In [18]:
# new_input의 capital, culture, climate, location가  template_total의 capital, culture, climate, location 변수와 연결된다
template_total = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}
location :  {location}

이 정보를 바탕으로 여행 가기 가장 좋은 계절을 알려주세요
"""
prompt_total  = PromptTemplate(input_variables=["capital", "culture", "climate", "location"],
                                 template=template_total)

In [19]:
travel_chain = new_input | prompt_total | llm | StrOutputParser()

In [20]:
travel_chain.invoke({"country" : "태국"})

'태국 여행을 계획하고 계시는군요! 제가 가진 정보를 바탕으로 말씀드리면, **11월부터 2월까지의 건기**가 태국 여행하기에 가장 좋은 시기입니다! 🌞\n\n## 왜 11월-2월이 최고인가요?\n\n**🌡️ 기온이 가장 쾌적해요**\n- 평균 기온이 20-30°C로 태국 기준에서는 상당히 시원한 편입니다\n- 특히 12월-1월은 연중 가장 시원해서 야외 활동하기 정말 좋아요\n\n**☀️ 날씨가 안정적이에요**\n- 비가 거의 오지 않아서 여행 일정이 비에 방해받을 걱정이 없어요\n- 맑은 날씨가 많아서 방콕의 왕궁이나 왓 포 사원 같은 야외 관광지 구경하기 완벽합니다\n\n**💨 습도가 상대적으로 낮아요**\n- 연중 70-80%인 습도가 이 시기에는 좀 더 견딜 만해져요\n- 방콕 시내를 BTS나 MRT로 이동할 때도 덜 끈적거려요\n\n## 지역별로도 차이가 있어요!\n- **치앙마이 같은 북부**는 이 시기에 정말 시원하고 건조해서 최고예요\n- **방콕**은 전형적인 열대기후지만 그래도 이때가 가장 나아요\n- **푸켓이나 코사무이** 같은 남부 해변도 이 시기가 베스트시즌이에요\n\n다만 이 시기가 성수기라서 항공료나 숙박비가 좀 더 비쌀 수 있다는 점은 참고하세요! 그래도 쾌적한 날씨를 생각하면 충분히 가치가 있다고 생각해요. 혹시 구체적으로 어느 지역을 가실 계획인가요? 🇹🇭'

In [21]:
#비슷한 방법으로 RunnableParallel도 존재한다 
parallel_chain = RunnableParallel(
                                  capital=capital_chain, 
                                  culture=culture_chain,
                                  climate=climate_chain, 
                                  location = climate_location
                                 )

In [22]:
travel_chain_parallel =  parallel_chain | prompt_total | llm | StrOutputParser()

In [23]:
travel_chain_parallel.invoke({"country" : "태국"})

'제가 가진 정보를 종합해보면, **태국 여행의 최적 시기는 11월부터 2월까지의 건기**입니다! 🌞\n\n## 왜 이 시기가 최고일까요?\n\n### 🌡️ 완벽한 날씨 조건\n- **기온**: 25-30°C로 가장 쾌적함\n- **습도**: 상대적으로 낮아서 덜 끈적거림\n- **강수량**: 거의 비가 오지 않아 야외활동 최적\n\n### 🏖️ 지역별 장점\n\n**방콕**: \n- 왕궁, 왓 포, 왓 아룬 등 야외 관광지 탐방하기 완벽\n- 차투착 시장이나 카오산 로드 돌아다니기 좋음\n\n**푸켓/끄라비**: \n- 파통 비치, 라일레이 비치에서 해수욕 즐기기 최고\n- 피피 섬, 홍 섬 호핑 투어하기 이상적\n\n**치앙마이**: \n- 도이 수텝 사원 올라가기 좋음 (북부라 밤에는 서늘해서 일교차 있음)\n- 올드시티 걸어다니며 구경하기 완벽\n\n**코사무이**: \n- 차웽 비치에서 수영하고 일광욕하기 최적\n- 나무앙 폭포 트레킹하기 좋음\n\n### 💡 추가 팁\n- 12월-1월이 가장 인기 시기라 숙박비가 비쌀 수 있어요\n- 11월이나 2월이 상대적으로 여행비용 절약하면서도 좋은 날씨 즐길 수 있습니다!\n\n태국의 미소 문화와 불교 사원들, 그리고 맛있는 팟타이와 똠얌꿍을 이 완벽한 날씨에 즐기시면 정말 환상적인 여행이 될 거예요! 🇹🇭✨'

In [24]:
# llm의 outpu의 경우 StrOutputParser()를 활용해서 string으로 나오지만 llm.with_structured_output(obj)을 통해 json포맷으로 받을수 있다 
# llm의 결과가 json포맷으로 오면 이를 활용하여 다른 langchain에 활용할수 있다

In [25]:
template_plan = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}
location :  {location}

이 정보를 바탕으로 여행 계획을 세워주세요
"""
prompt_plan  = PromptTemplate(input_variables=["capital", "culture", "climate", "location"],
                                 template=template_plan)

In [26]:
#llm의 결과값 포맷의 schema
class Travel(BaseModel):
    capital: str = Field(description="국가의 수도에 대한 설명")
    culture: str = Field(description="국가의 문화에 대한 설명")
    climate : str = Field(description="국가 기후에 대한 설명")
    location : str = Field(description="국가 관광명소에 대한 설명")
    plan: str = Field(description="여행계획")


In [27]:
one_chain = (
    {
                "capital" :  itemgetter("country") | capital_chain,
                "culture" :  itemgetter("country") | culture_chain,
                "climate" :  itemgetter("country") | climate_chain,
                "location":  itemgetter("country") | climate_chain
    }
    | prompt_plan
    | llm.with_structured_output(Travel)
)
 

In [28]:
one_chain.invoke({"country" : "태국"})

Travel(capital='태국의 수도는 **방콕(Bangkok)**입니다. 정식 명칭은 크룽텝마하나콘이며, 인구 약 1,000만 명의 대도시입니다. 태국의 정치·경제·문화의 중심지로, 왕궁, 왓 포 사원, 왓 아룬 사원, 카오산 로드 등 주요 관광명소가 있습니다. 스카이트레인(BTS), 지하철(MRT), 보트 등 다양한 교통수단이 발달되어 있으며, 차오프라야 강이 도시를 관통해 "동양의 베니스"라고도 불립니다.', culture='태국은 인구의 95%가 소승불교도인 불교 국가로, 일상생활에 불교 철학이 깊이 스며들어 있습니다. 와이(Wai)라는 전통 인사법을 사용하며, 연장자와 승려에 대한 깊은 존경 문화가 있습니다. "미소의 나라"라 불릴 만큼 온화한 성격의 사람들이 많습니다. 쌀을 주식으로 하며 톰얌꿍, 팟타이, 그린커리 등의 유명 요리와 발달된 길거리 음식 문화를 가지고 있습니다. 무에타이, 전통 무용, 전통 음악 등의 예술 문화와 왕실에 대한 절대적 존경 문화도 특징적입니다.', climate='태국은 열대성 기후로 연중 고온다습합니다. 평균 기온은 25-35°C이며 습도는 70-80%로 높습니다. 건기(11월-2월)는 가장 여행하기 좋은 시기로 기온 25-30°C, 더운 건기(3월-5월)는 가장 더운 시기로 30-40°C, 우기(6월-10월)는 몬순 시즌으로 25-32°C에 거의 매일 스콜성 비가 내립니다. 북부는 상대적으로 서늘하고, 중부는 전형적인 열대기후, 남부는 연중 고온다습한 특징을 보입니다.', location='태국의 주요 관광명소로는 방콕의 왕궁, 왓 포 사원, 왓 아룬 사원, 카오산 로드가 있습니다. 북부 치앙마이와 치앙라이는 상대적으로 서늘한 기후를 가지고 있으며, 중부 방콕은 전형적인 열대기후 지역입니다. 남부의 푸켓과 코사무이는 연중 고온다습한 해변 휴양지로 유명합니다. 차오프라야 강이 방콕을 관통하며 보트 교통이 발달되어 있고, 전국에 4만여 개의 불교 사원이 분포되어 있습니다.', plan='태국 여행 최적 시기는 11월-

In [29]:
#data는 llm의 결과 값을 의미합니다  RunnableLambda함수를 활용하면 data와 llm결과를 연결 시켜줄수 있습니다
def transform_keys(data):
    """Transform keys from history to chat_history"""
    return {
        "capital":  data.capital,
        "culture":  data.culture,  
        "climate":  data.climate, 
        "location": data.location, 
        "plan":     data.plan
    }

In [30]:
template_detail = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}
location :  {location}
plan :      {plan}

plan에 담겨있는 관광지 혹은 축제에 대한 정보를 좀더 상세하게 알려두세요 
"""
prompt_detail  = PromptTemplate(input_variables=["capital", "culture", "climate", "location", "plan"],
                                 template=template_detail)

In [31]:
# RunnableLambda(transform_keys)는 llm과 prompt_detail의 데이터를 매핑 시켜주는 역할
chain_and_chain = (
    {
                "capital" :  itemgetter("country") | capital_chain,
                "culture" :  itemgetter("country") | culture_chain,
                "climate" :  itemgetter("country") | climate_chain,
                "location":  itemgetter("country") | climate_chain
    }
    | prompt_plan
    | llm.with_structured_output(Travel)
    | RunnableLambda(transform_keys)
    | prompt_detail
    | llm
    | StrOutputParser()
)


In [32]:
chain_and_chain.invoke({"country" : "일본"})

"일본 여행 계획에 포함된 관광지와 문화 체험에 대해 더 자세히 설명해드리겠습니다!\n\n## 🏙️ **도쿄 주요 관광지**\n\n**신주쿠**\n- 도쿄도청 전망대: 무료로 이용 가능한 202m 높이 전망대에서 도쿄 전경과 날씨가 좋으면 후지산까지 볼 수 있습니다\n- 가부키초: 일본 최대 환락가로 네온사인이 화려한 밤 문화의 중심지\n- 신주쿠 교엔: 58.3헥타르 규모의 대형 공원으로 벚꽃 시즌에는 1,500그루의 벚나무가 만개합니다\n\n**아사쿠사**\n- 센소지 절: 645년 창건된 도쿄에서 가장 오래된 사찰로, 가미나리몬(雷門)의 거대한 빨간 등롱이 상징적입니다\n- 나카미세도리: 센소지로 이어지는 250m 상점가에서 전통 과자와 기념품을 판매합니다\n\n**시부야**\n- 시부야 스크램블 교차로: 한 번에 최대 3,000명이 건너는 세계에서 가장 바쁜 교차로\n- 하치코 동상: 충견 하치코를 기리는 동상으로 유명한 만남의 장소입니다\n\n## 🏛️ **교토 전통 문화**\n\n**주요 사찰**\n- 기요미즈데라: 778년 창건, 나무못 하나 사용하지 않고 지은 본당에서 교토 시내 전망 감상\n- 후시미 이나리 신사: 1만 개가 넘는 주황색 도리이가 산길을 따라 이어지는 장관\n- 긴카쿠지(은각사): 무로마치 시대 건축의 걸작으로 정교한 일본 정원과 조화\n\n**전통 체험**\n- 기모노 렌탈: 기온 지구에서 전통 기모노를 입고 돌담길과 전통 가옥 사이를 거닐 수 있습니다\n- 대나무 숲(아라시야마): 수천 그루의 대나무가 만드는 초록 터널로 바람소리가 일본 환경성 선정 '보존하고 싶은 일본의 소리 100선'에 포함\n\n## 🏯 **오사카 명소**\n\n**오사카성**\n- 1583년 도요토미 히데요시가 건설한 5층 천수각\n- 벚꽃 시즌에는 4,000그루의 벚나무로 유명한 하나미 명소\n- 성 내부는 박물관으로 일본 전국시대 역사를 전시\n\n## 🍜 **음식 문화 체험**\n\n**지역별 특색 음식**\n- 도쿄: 에도마에 스시(신선한 생선

In [33]:
#사용했던 모든 데이터 출력
# result=await extract_stream(chain_and_chain, {"country" : "태국"})

In [50]:
template_plan_final = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}

이 정보를 바탕으로 여행 계획을 세워주세요
"""
prompt_plan_final  = PromptTemplate(input_variables=["capital", "culture", "climate"],
                                 template=template_plan_final)

In [51]:
class TravelLast(BaseModel):
    capital: str = Field(description="국가의 수도에 대한 설명")
    culture: str = Field(description="국가의 문화에 대한 설명")
    climate : str = Field(description="국가 기후에 대한 설명")
    plan: str = Field(description="여행계획")

In [52]:
def transform_key_last(data):
    """Transform keys from history to chat_history"""
    return {
        "capital":  data.capital,
        "culture":  data.culture,  
        "climate":  data.climate, 
        "plan":     data.plan
    }

In [53]:
template_detail_final = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

capitial :  {capital}
culture :   {culture}
climate :   {climate}
plan :      {plan}

plan에 담겨있는 관광지 혹은 축제에 대한 정보를 좀더 상세하게 알려두세요 
"""
prompt_detail_final = PromptTemplate(input_variables=["capital", "culture", "climate", "plan"],
                                 template=template_detail_final)

In [54]:
two_json_chain=({
    "capital": itemgetter("country") | capital_chain,
    "culture": itemgetter("country") | culture_chain,
    "climate": itemgetter("country") | climate_chain
    }
    | prompt_plan_final
    | llm.with_structured_output(TravelLast)
    | RunnableLambda(transform_key_last)
    | {
        "capital": itemgetter("capital"),   # capital("country") -> itemgetter("capital")
        "culture": itemgetter("culture"),   # 쉼표 추가
        "climate": itemgetter("climate"),   # 쉼표 추가
        "plan": itemgetter("plan")
    }
    | prompt_detail_final
    | llm
    | StrOutputParser()
    )


In [55]:
two_json_chain.invoke({"country" : "중국"})

"안녕하세요! 중국 여행 계획에 포함된 관광지와 축제들에 대해 상세히 알려드리겠습니다.\n\n## **베이징 주요 관광지**\n\n**천안문광장 (天安门广场)**\n- 세계에서 가장 큰 광장 중 하나로 면적이 44만㎡에 달합니다\n- 1949년 마오쩌둥이 중화인민공화국 건국을 선포한 역사적 장소\n- 인민영웅기념비, 마오주석기념당, 인민대회당이 위치\n- 매일 일출/일몰 시 국기 게양/하강식이 거행됩니다\n\n**자금성 (紫禁城)**\n- 명·청 황제들의 궁궐로 1420년 완공, 약 600년 역사\n- 9,999개 반의 방을 가진 세계 최대 규모의 목조 건축군\n- 태화전, 중화전, 보화전의 삼대전과 황제의 침전인 건청궁이 유명\n- 입장료: 성인 60위안(비수기), 80위안(성수기)\n\n**만리장성 팔달령 구간**\n- 베이징에서 약 70km 떨어진 가장 잘 보존된 구간\n- 해발 1,015m 높이에서 웅장한 산맥 전망 감상 가능\n- 케이블카 이용 시 편리하지만 도보 등반도 추천\n- 소요시간: 왕복 6-8시간 (이동시간 포함)\n\n**천단공원 (天坛公园)**\n- 명·청 황제들이 하늘에 제사를 지내던 제단\n- 원형의 기년전(祈年殿)이 대표적 건축물\n- 음향 효과가 뛰어난 회음벽(回音壁)으로 유명\n- 이른 아침 태극권하는 현지인들과 함께 운동 가능\n\n## **상하이 주요 관광지**\n\n**와이탄 (外滩)**\n- 황푸강변 1.5km 구간의 서구식 건축물군\n- 1920-30년대 '동양의 파리'라 불렸던 조계지 시절 건물들\n- 강 건너편 푸둥의 현대적 스카이라인과 대조되는 야경이 장관\n- 최적 관람시간: 오후 7-9시 (조명이 켜지는 시간)\n\n**예원 (豫园)**\n- 명나라 시대(1559년) 조성된 전통 중국식 정원\n- 400년 역사의 고전적 강남 정원 양식\n- 인근 예원상가에서 샤오롱바오(小笼包) 등 상하이 전통 음식 맛볼 수 있음\n- 입장료: 40위안, 관람시간: 2-3시간\n\n## **시안 주요 관광지**\n\n**병마용 